# Politics in Georgia

In [1]:
SCHOOL="georgia"
SUBJECT="opinions"
START_YEAR=2010
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%cd ..

/home/alexandersuen/diversity-colleges


In [3]:
%cd src

/home/alexandersuen/diversity-colleges/src


In [4]:
import sys
sys.path.append('src')
import ouraws
import nltk
import ourgraphs
import textutil

In [5]:
%cd ..

/home/alexandersuen/diversity-colleges


In [6]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [7]:
df.shape

(1535, 6)

In [8]:
df.head(2)

,title,url,body,year,month,day
0,Minimalism is damaging for the world of design,https://nique.net/opinions/2022/11/14/minimali...,Minimalism is damaging for the world of design...,2022,11,14
1,Working a job in high school,https://nique.net/opinions/2022/11/14/working-...,Working a job in high school\nOf all the exper...,2022,11,14


In [9]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")
#BIPARTISAN_API_KEY = "gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=="
print(BIPARTISAN_API_KEY)
BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g==


In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-4.84244


In [11]:
df['year'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009])

In [12]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2010 ... 141
2010	137	-159.02218600000003	-1.160745883211679
Year: 2011 ... 150
2011	150	60.117597000000025	0.40078398000000015
Year: 2012 ... 101
2012	98	-37.24838399999999	-0.38008555102040803
Year: 2013 ... 139
2013	138	-46.122153	-0.3342185
Year: 2014 ... 124
2014	121	30.055629999999987	0.24839363636363626
Year: 2015 ... 118
2015	113	-61.84941900000002	-0.5473399911504426
Year: 2016 ... 135
2016	132	-288.7457630000001	-2.1874679015151526
Year: 2017 ... 158
2017	153	-367.5125910000001	-2.4020430784313733
Year: 2018 ... 92
2018	86	-190.98033899999996	-2.220701616279069
Year: 2019 ... 79
2019	76	-158.48872699999995	-2.0853779868421047
Year: 2020 ... 76
2020	75	-201.636834	-2.68849112
Year: 2021 ... 97
2021	97	-141.32442799999998	-1.4569528659793813
Year: 2022 ... 116
2022	115	-162.85228299999997	-1.4161068086956519


In [13]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)

In [ ]:
# ASYNCHRONOUS METHOD : should be a lot faster
import asyncio

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year